In [1]:
import pandas as pd

In [2]:
from pycparser import c_parser, c_ast
import pandas as pd
import os
import re
import sys
from gensim.models.word2vec import Word2Vec
import pickle
from tree import ASTNode, SingleNode
import numpy as np


def get_sequences(node, sequence):
    current = SingleNode(node)
    sequence.append(current.get_token())
    for _, child in node.children():
        get_sequences(child, sequence)
    if current.get_token().lower() == 'compound':
        sequence.append('End')


def get_blocks(node, block_seq):
    children = node.children()
    name = node.__class__.__name__
    if name in ['FuncDef', 'If', 'For', 'While', 'DoWhile']:
        block_seq.append(ASTNode(node))
        if name is not 'For':
            skip = 1
        else:
            skip = len(children) - 1

        for i in range(skip, len(children)):
            child = children[i][1]
            if child.__class__.__name__ not in ['FuncDef', 'If', 'For', 'While', 'DoWhile', 'Compound']:
                block_seq.append(ASTNode(child))
            get_blocks(child, block_seq)
    elif name is 'Compound':
        block_seq.append(ASTNode(name))
        for _, child in node.children():
            if child.__class__.__name__ not in ['If', 'For', 'While', 'DoWhile']:
                block_seq.append(ASTNode(child))
            get_blocks(child, block_seq)
        block_seq.append(ASTNode('End'))
    else:
        block_seq.append(ASTNode(name)) # 增加一个头
        for _, child in node.children():
            get_blocks(child, block_seq)


In [3]:
blocks = pd.read_pickle('blocks.pkl')

In [4]:
tmp = list(blocks['code'])[0]

In [7]:
blocks

,id,code,label
46887,46887,None,93
4970,4970,None,42
50521,50521,None,77
38406,38406,None,63
44230,44230,None,80
21122,21122,None,69
31463,31463,None,84
43363,43363,None,88
16692,16692,None,7
9937,9937,None,8


In [5]:
tmp

In [6]:
from gensim.models.word2vec import Word2Vec

In [7]:
word2vec = Word2Vec.load('embedding/node_w2v_128').wv

In [8]:
word2vec.vocab

{'FileAST': <gensim.models.keyedvectors.Vocab at 0x1b61824e828>,
 'FuncDef': <gensim.models.keyedvectors.Vocab at 0x1b61824eb38>,
 'Decl': <gensim.models.keyedvectors.Vocab at 0x1b61824ecc0>,
 'FuncDecl': <gensim.models.keyedvectors.Vocab at 0x1b61824e8d0>,
 'main': <gensim.models.keyedvectors.Vocab at 0x1b61824ee80>,
 'void': <gensim.models.keyedvectors.Vocab at 0x1b61824ec50>,
 'Compound': <gensim.models.keyedvectors.Vocab at 0x1b61824e9b0>,
 'a': <gensim.models.keyedvectors.Vocab at 0x1b61824eba8>,
 'int': <gensim.models.keyedvectors.Vocab at 0x1b61824e780>,
 'i3': <gensim.models.keyedvectors.Vocab at 0x1b61824e898>,
 'i5': <gensim.models.keyedvectors.Vocab at 0x1b61824e668>,
 'i7': <gensim.models.keyedvectors.Vocab at 0x1b61824ec88>,
 'FuncCall': <gensim.models.keyedvectors.Vocab at 0x1b61824e6d8>,
 'scanf': <gensim.models.keyedvectors.Vocab at 0x1b61824ed30>,
 'ExprList': <gensim.models.keyedvectors.Vocab at 0x1b61824e9e8>,
 '"%d"': <gensim.models.keyedvectors.Vocab at 0x1b61824e9

In [9]:
tmp_rst = []
def list2word(l):
    if type(l) == type([]):
        for e in l:
            list2word(l)
    else:
        l = word2vec.index2word[l]
list2word(tmp)

RecursionError: maximum recursion depth exceeded while calling a Python object

In [10]:
train_pkl = pd.read_pickle('train_.pkl')

In [11]:
train_pkl

,id,code,label
46887,46887,"FileAST(ext=[FuncDef(decl=Decl(name='main',\n ...",93
4970,4970,"FileAST(ext=[FuncDef(decl=Decl(name='main',\n ...",42
50521,50521,"FileAST(ext=[Decl(name='i',\n ...",77
38406,38406,"FileAST(ext=[FuncDef(decl=Decl(name='main',\n ...",63
44230,44230,"FileAST(ext=[Decl(name='DiJiTian',\n ...",80
21122,21122,"FileAST(ext=[FuncDef(decl=Decl(name='f',\n ...",69
31463,31463,"FileAST(ext=[FuncDef(decl=Decl(name='main',\n ...",84
43363,43363,"FileAST(ext=[FuncDef(decl=Decl(name='main',\n ...",88
16692,16692,"FileAST(ext=[FuncDef(decl=Decl(name='main',\n ...",7
9937,9937,"FileAST(ext=[FuncDef(decl=Decl(name='get',\n ...",8


In [12]:
train_pkl0 = list(train_pkl['code'])[0]

In [16]:
train_pkl0

FileAST(ext=[FuncDef(decl=Decl(name='main',
                               quals=[
                                     ],
                               storage=[
                                       ],
                               funcspec=[
                                        ],
                               type=FuncDecl(args=None,
                                             type=TypeDecl(declname='main',
                                                           quals=[
                                                                 ],
                                                           type=IdentifierType(names=['void'
                                                                                     ]
                                                                               )
                                                           )
                                             ),
                               init=None,
                              

In [21]:
block_tmp = []
train_pkl0_block = get_blocks(train_pkl0,block_tmp)

In [22]:
len(block_tmp)

109

In [31]:
train_pkl0.__class__.__name__

'FileAST'